In [1]:
import xarray as xr
import plotly.graph_objects as go
import pandas as pd
import cf_xarray.units  # noqa: F401
import pint_xarray  # noqa: F401
from datetime import datetime
from pathlib import Path

pd.options.plotting.backend = "plotly"

In [2]:
station_name = "HOT"
latitude = 22.75
longitude = -158
release_folder = "../../../../release/"

---

## Load datasets


In [3]:
data_hot = xr.load_dataset("../../3_post_processed/hot_product.nc")
data_hot

<xarray.Dataset> Size: 4MB
Dimensions:     (time: 687, is_day: 2, latitude: 1, longitude: 1, depth: 8,
                 sieve_size: 5)
Coordinates:
  * time        (time) datetime64[ns] 5kB 1988-10-31 1988-12-02 ... 2022-09-02
  * is_day      (is_day) bool 2B False True
  * latitude    (latitude) float64 8B 22.75
  * longitude   (longitude) int64 8B -158
  * depth       (depth) int64 64B 50 100 150 200 250 300 350 400
  * sieve_size  (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
Data variables: (12/25)
    vol         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    svol        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    wwt         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    dwt         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    carb        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    nitrogen    (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    ...          ...
    temp        (time, latitude, longitude, depth) float64 44kB 26.21 ... nan
    sal         (time, latitude, longitude, depth) float64 44kB 35.21 ... nan
    oxy         (time, latitude, longitude, depth) float64 44kB 217.5 ... nan
    xmiss       (time, latitude, longitude, depth) float64 44kB 3.42 ... nan
    fluor       (time, latitude, longitude, depth) float64 44kB 0.1021 ... nan
    nitrate     (time, latitude, longitude, depth) float64 44kB nan nan ... nan

---

## Vizualization


In [4]:
zoo_hot = (
    data_hot["dwt"]
    .dropna("time", how="all")
    .mean("depth")
    .isel(latitude=0, longitude=0)
    .sum("sieve_size")
)
hot_units = data_hot["dwt"].pint.quantify().pint.units
zoo_hot = zoo_hot.pint.quantify(hot_units).pint.to("mg/m^3").pint.dequantify()
zoo_hot_d_n = zoo_hot.to_dataframe().pivot_table(
    index=["time"], columns=["is_day"], values=["dwt"]
)
zoo_hot_d_n.columns = ["night", "day"]
# # transform 0 to nan
zoo_hot_d_n = zoo_hot_d_n.map(lambda x: None if x == 0 else x)

In [5]:
day = zoo_hot_d_n["day"].dropna()
night = zoo_hot_d_n["night"].dropna()
go.Figure(
    data=[
        go.Scatter(x=day.index, y=day, mode="lines", name="Day"),
        go.Scatter(x=night.index, y=night, mode="lines", name="Night"),
    ]
).update_layout(
    title="hot : zooplankton dry weight by cube meter of sea water",
    xaxis_title="Time",
    yaxis_title="Dry Weight (mg/m3)",
    legend_title="Time of Day",
).show()

---

## Export the products in netCDF format

First, write the function to prepare the forcing and save it as NetCDF file.


In [6]:
as_xarray = xr.Dataset.from_dataframe(zoo_hot_d_n)
as_xarray = as_xarray.expand_dims(
    {"latitude": [latitude], "longitude": [longitude], "layer": [0]}
)
as_xarray["time"].attrs = {
    "axis": "T",
    "standard_name": "time",
    "long_name": "time",
}
as_xarray["night"].attrs = {
    "units": "mg/m3",
    "description": "Zooplankton dry weight by cube meter of sea water at night",
}
as_xarray["day"].attrs = {
    "units": "mg/m3",
    "description": "Zooplankton dry weight by cube meter of sea water during the day",
}
as_xarray.attrs = {
    "station": station_name,
    "creation date": datetime.now().isoformat(),
}
as_xarray["layer"].attrs = {
    "flag_values": [0, 1, 2],
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "layer",
    "long_name": "Layer depth",
    "axis": "Z",
}
as_xarray["latitude"].attrs = {
    "standard_name": "latitude",
    "axis": "Y",
    "units": "degrees_north",
    "long_name": "Latitude",
}
as_xarray["longitude"].attrs = {
    "standard_name": "longitude",
    "axis": "X",
    "units": "degrees_east",
    "long_name": "Longitude",
}
as_xarray = as_xarray.transpose("time", "latitude", "longitude", "layer")
as_xarray = as_xarray.chunk({"time": -1})
as_xarray.to_zarr(Path(release_folder) / f"{station_name}_zooplankton.zarr", mode="w")